In [5]:
from h5py import File
from pathlib import Path
import numpy as np

album_path = Path('/data/i3store/users/ssued/albums/RNO_album_10_12_2025')


with File(album_path / 'n1_album' / 'n1_album.hdf5') as f1:
    n1 = len(f1.keys())

with File(album_path / 'n2_album' / 'n2_album.hdf5') as f2:
    n2 = len(f2.keys())

with File(album_path / 'n3_album' / 'n3_album.hdf5') as f3:
    n3 = len(f3.keys())

try:
    with File(album_path / 'n4_album' / 'n4_album.hdf5') as f4:
        n4 = len(f4.keys())
except Exception as e:
    print('No n4 album found')
    n4 = 0

total = n1 + n2 + n3 + n4
print(f"n1: {n1}, n2: {n2}, n3: {n3}. n4: {n4}, total: {total}")
print(f"Ratios: n1/total = {n1/total * 100:.4f}%, n2/total = {n2/total * 100:.4f}%, n3/total = {n3/total * 100:.4f}%,  n4/total = {n4/total * 100:.4f}%")

No n4 album found
n1: 38101, n2: 2990, n3: 20. n4: 0, total: 41111
Ratios: n1/total = 92.6784%, n2/total = 7.2730%, n3/total = 0.0486%,  n4/total = 0.0000%


# Obtain maximum voltage:

In [8]:
from IPython.display import clear_output
import time

album_path = Path('/data/i3store/users/ssued/albums/RNO_album_10_10_2025')

with File(album_path / 'album_RNO4.hdf5') as f:
    v_max = -1
    for i in range(1249,len(f.keys())):
        clear_output(wait=False)
        print(f'Looking at event{i+1}/{len(f.keys())}')
        print(f'Current max: {v_max} V')
        img = f[f'event{i+1}']['image']
        v_max_curr = np.max(img)
        print(f'Comparing Maximum ({v_max}) with Current Max ({v_max_curr})')
        if v_max < v_max_curr:
            v_max = v_max_curr
        time.sleep(5)

        # 1253

Looking at event1253/70111
Current max: 7273.981606603524 V
Comparing Maximum (7273.981606603524) with Current Max (5.427471675803142)


KeyboardInterrupt: 

# To visualize Images:

In [10]:
with File(album_path / album_name) as f:
    max = np.max(f['event1252']['image'])

    print(max)

7273.981606603524


In [ ]:
from ipywidgets import interact, widgets, interactive_output
from h5py import File
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


album_path = Path('/data/i3store/users/ssued/albums/RNO_album_10_09_2025')
album_name = 'album_RNO4.hdf5'
num_keys = 0

with File(album_path / album_name) as f:
    num_keys = len(f.keys())

def view_image(i=1):
    with File(album_path / album_name) as f:
        data = f[f'event{i}']['image'][:]
        fig = make_subplots(rows=4, cols=1)
        
        for iax in range(4):
            fig.add_trace(
                go.Heatmap(z=data[:, :, iax], colorscale='Viridis', hovertemplate='x: %{x}<br>y: %{y}<br>value: %{z}'),
                row=iax+1, col=1
            )
        
        fig.update_layout(
            title_text=f'Image {i}/{num_keys}', 
            height=1000,
            width=2000
        )
        
        fig.update_xaxes(showticklabels=True)
        fig.update_yaxes(showticklabels=True)
        
        fig.show()

event_slider = widgets.IntText(value=1, min=1, max=num_keys, description='Event:')
output = interactive_output(view_image, {'i': event_slider})
display(event_slider, output)

IntText(value=1, description='Event:')

Output()

In [ ]:
with File(album_path / album_name) as f:
    image = f['event1']['image']
    non_zero_count = np.count_nonzero(image)
    print(f"Number of non-zero entries in image: {non_zero_count} / {image.size} ({non_zero_count/image.size * 100:.3f}%)")

Number of non-zero entries in image: 1752 / 98304 (1.782%)


# For moving data around...

In [ ]:
from h5py import File
from pathlib import Path
import os
import sys
sys.path.append('/data/condor_shared/users/ssued/RNOG_Image_Builder')

# from utils.my_utils import find_max_hdf5_index



album_path = Path('/data/i3store/users/ssued/albums/RNO_album_10_10_2025/n4_album')
album_name = 'n4_album.hdf5'
source_album_path = album_path / album_name
dest_album_path = Path('/data/i3store/users/ssued/albums/RNO_benchmark.hdf5')

# Open both source and destination HDF5 files
# 'r' mode: read-only access to source file
# 'a' mode: append/read-write access to master file (creates if doesn't exist)
with (File(source_album_path, 'r') as h5_source, 
      File(dest_album_path, 'a') as h5_destination):
    
    print(f'Copying events from {h5_source} to {h5_destination}')

    h5_source.copy(h5_source['event1'], h5_destination, 'event4')

    

Copying events from <HDF5 file "n4_album.hdf5" (mode r)> to <HDF5 file "RNO_benchmark.hdf5" (mode r+)>
